In [6]:
%load_ext autoreload
%autoreload 2

In [8]:
import numpy as np
import nibabel as nib
import glob 
import os
from scipy.ndimage import binary_dilation
from ni_functions import get_atlas_coords, image_overlaps

In [9]:
project_path = "/workspaces/bha2/data/craddock/craddock_115"
brain_areas = ['Frontal', 'Parietal', 'Occipital', 'Temporal', 'Insula',  'Subcortical']

In [180]:
template_empty = np.zeros(nib.load(os.path.join('/workspaces', 'bha2', 'brain_templates', 'lobule_masks', 'Frontal_mask.nii.gz')).shape)
volume_per_roi_desired = 115
min_volume_per_roi = 30

In [181]:
final_roi_label_counter = 1
for b_area in brain_areas:
    image_names = glob.glob(os.path.join(project_path, b_area, '*.nii.gz'))

    image_roi_labels = []
    image_roi_volumes = []
    image_mean_roi_vol = []

    for im_name in image_names:
        image = nib.load(im_name)
        image_data = image.get_fdata()
        rois = np.setdiff1d(np.unique(image_data), 0).astype(int)

        roi_vol = np.zeros((len(rois)))
        for idx, roi_number in enumerate(rois):
            roi_vol[idx] = np.sum(image_data == roi_number)
        image_roi_volumes.append(roi_vol)
        image_mean_roi_vol.append(np.mean(roi_vol))
        image_roi_labels.append(rois)

    size_near_desired = np.argmin(
        np.abs(np.array(image_mean_roi_vol) - volume_per_roi_desired))
    image_roi_vol_selected = image_roi_volumes[size_near_desired]
    image_selected = nib.load(image_names[size_near_desired])
    small_rois = image_roi_labels[size_near_desired][np.where(
        image_roi_vol_selected < min_volume_per_roi)[0]]

    vol_for_small_rois_correction = image_selected.get_fdata()
    if len(small_rois) > 0:
        for sr in small_rois:
            sr_loc = np.where(vol_for_small_rois_correction ==
                              sr, 1, 0).astype(bool)
            sr_dil_loc = binary_dilation(sr_loc, iterations=1)
            neighbours = np.setdiff1d(
                np.unique(vol_for_small_rois_correction[sr_dil_loc]), [0, sr]).astype(int)
            if len(neighbours) == 0:
                sr_dil_loc = binary_dilation(sr_loc, iterations=2)
                neighbours = np.setdiff1d(
                    np.unique(vol_for_small_rois_correction[sr_dil_loc]), [0, sr]).astype(int)
            if len(neighbours) > 0:
                size_neighbours = image_roi_vol_selected[np.where(
                    np.in1d(image_roi_labels[size_near_desired], neighbours))[0]]
                small_neighbour_idx = np.argmin(size_neighbours)
                vol_for_small_rois_correction[sr_loc] = neighbours[small_neighbour_idx]
            else:
                vol_for_small_rois_correction[sr_loc] = 0

    new_roi_labels = np.setdiff1d(
        np.unique(vol_for_small_rois_correction), 0).astype(int)
    for roi in new_roi_labels:
        template_empty[vol_for_small_rois_correction ==
                       roi] = final_roi_label_counter
        final_roi_label_counter += 1

nib.save(nib.Nifti1Image(template_empty, image_selected.affine), os.path.join(
    '/workspaces', 'bha2', 'brain_templates', 'craddock_' + str(final_roi_label_counter-1) + '.nii.gz'))


In [21]:
atlas_path = os.path.join('/workspaces', 'bha2',
                          'brain_templates', 'craddock_' + str(final_roi_label_counter-1) + '.nii.gz')
atlas_coords = get_atlas_coords(atlas_path)

atlas = nib.load(atlas_path)
atlas_data = atlas.get_fdata()
atlas_rois = np.setdiff1d(np.unique(atlas_data), 0).astype(int)

overlaps_with_brain_lobes = np.zeros((len(atlas_rois), 2*len(brain_areas)))
idx_b_area = 0
for b_area in brain_areas:
    for roi in atlas_rois:
        b_area_mask_L = nib.load(os.path.join('/workspaces', 'bha2', 'brain_templates',
                               'lobule_masks', 'masks_splitted_in_hemispheres', b_area + '_L_mask.nii.gz'))
        b_area_mask_L_data = b_area_mask_L.get_fdata()
        b_area_mask_R = nib.load(os.path.join('/workspaces', 'bha2', 'brain_templates',
                               'lobule_masks', 'masks_splitted_in_hemispheres', b_area + '_R_mask.nii.gz'))
        b_area_mask_R_data = b_area_mask_R.get_fdata()
        roi_mask = np.where(atlas_data == roi, 1, 0)
        atlas_coords.loc[roi, b_area + '_L'] = image_overlaps(roi_mask, b_area_mask_L_data)
        atlas_coords.loc[roi, b_area + '_R'] = image_overlaps(roi_mask, b_area_mask_R_data)

    idx_b_area += 2

atlas_coords.index.name = 'ROI_number'
atlas_coords.to_csv(os.path.join('/workspaces', 'bha2', 'brain_templates', 'craddock_' + str(final_roi_label_counter-1) + '_rois.csv'))